In [1]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime as dt

import seaborn as sns
import datetime

##DataFrames To be used for analysis of infielders
#Reguler_Catcher_Stats_30.csv   *played atleast 30 games
#Reguler_Catcher_Stats_100.csv  *played atleast 100 games
#All_Star_Catcher_Stats_100.csv  *alstars who played atleast 100 gms



In [33]:
import psycopg2 as p2
conn = p2.connect(database = 'player_stats', user ='baseball_master', password = 'georgetowndata', 
                  host = 'georgetown-baseball.cnfqonxqdbry.us-east-1.rds.amazonaws.com', 
                  port = '5432')

### Non- All Star Catchers who have played at least 30 games at their position

In [3]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'C'
and "G" > 30 and "G" < 100
and "playerID" not in 
   (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

Reguler_Catcher_30 = pd.read_sql_query(query, conn)

NameError: name 'conn' is not defined

In [ ]:
Reguler_Catcher_30.head(3)

### Batting Stats for Non All Stars Catchers who played atleast 30 games

In [ ]:
query="""
select a."playerID", m."nameFirst", m."nameLast", m."birthYear", m."deathYear", a."yearID", a."G", a."teamID", a."AB",a."H",a."2B", a."3B",
a."HR", a."SO", a."HBP", a."BB", a."SF"
from batting a
join master m
on a."playerID" = m."playerID"
where a."yearID" >= '1947'
"""

Batting_Stats = pd.read_sql_query(query,conn)

In [ ]:
Batting_Stats['AVG'] = Batting_Stats['H'] / Batting_Stats['AB']

In [ ]:
Batting_Stats["1B"] = Batting_Stats["H"] - Batting_Stats["2B"]- Batting_Stats["3B"]
- Batting_Stats["HR"]

In [ ]:
Batting_Stats.HBP.fillna(0, inplace = True)  #replace NaN with zero

In [ ]:
Batting_Stats.SF.fillna(0, inplace = True)  #replace NaN with zero

In [ ]:
Batting_Stats["TB"] = Batting_Stats["1B"] + Batting_Stats["2B"]*2 
+ Batting_Stats["3B"]*3 + Batting_Stats["HR"] * 4

In [ ]:
Batting_Stats["SLG"] = Batting_Stats["TB"] / Batting_Stats["AB"]

In [ ]:
Batting_Stats["OBP"] = (Batting_Stats["H"] + Batting_Stats["BB"] 
+ Batting_Stats["HBP"]) / (Batting_Stats ["AB"] + Batting_Stats ["BB"]
+ Batting_Stats["HBP"] + Batting_Stats["SF"])

In [ ]:
Batting_Stats.head(3)

In [ ]:
Batting_Stats["OPS"] = Batting_Stats["OBP"] + Batting_Stats["SLG"]

### Merge Catcher Dataframe with Batting Avg Dataframe

In [ ]:
Reguler_Catcher_Stats_30= pd.merge(Reguler_Catcher_30, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
Reguler_Catcher_Stats_30.head(3)

In [ ]:
Reguler_Catcher_Stats_30.to_csv('Reguler_Catcher_Stats_30.csv')

### Non- All Star Catcher who have played at least 100 games at their position and Batting Stats

In [ ]:
query="""

select "playerID", "POS", "yearID", "GS","G"
from fielding 
where "G" > 100
and "yearID" >= '1947'
and "POS" = 'C'
and "playerID" not in 
	(Select "playerID"  ---count("playerID")
from allstar_full
group by "playerID"
having count("playerID") > 5)
"""

Reguler_Catcher_100 = pd.read_sql_query(query, conn)

In [ ]:
Reguler_Catcher_Stats_100 = pd.merge(Reguler_Catcher_100, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
Reguler_Catcher_Stats_100.head(3)

In [ ]:
Reguler_Catcher_Stats_100.to_csv('Reguler_Catcher_Stats_100.csv')

### All Star Catchers who have plaved in over 5 All Star games and over 100 games

In [ ]:
query="""
select "playerID", "POS", "yearID", "GS", "G"
from fielding 
where "yearID" >= '1947'
and "POS" = 'C'
and "G" > 100
and "playerID" in  
        (Select "playerID"  ---count("playerID")
        from allstar_full
        group by "playerID"
        having count("playerID") > 5)
"""

All_Star_Catcher_100 = pd.read_sql_query(query, conn)

In [ ]:
All_Star_Catcher_Stats_100 = pd.merge(All_Star_Catcher_100, Batting_Stats, on = ["playerID", "yearID"])

In [ ]:
All_Star_Catcher_Stats_100.head(3)

In [ ]:
All_Star_Catcher_Stats_100.to_csv('All_Star_Catcher_Stats_100.csv')